### Load preprocessed data

In [0]:
import os.path
import numpy as np
import pandas as pd
import requests
from zipfile import ZipFile

In [3]:
!rm -rdf loader* data/* data*
!wget https://raw.githubusercontent.com/cemoody/simple_mf/master/notebooks/loader.py
!mkdir data
!wget https://www.dropbox.com/s/msqzmmaog5bstm4/dataset.npz?dl=0
!mv dataset.npz?dl=0 data/dataset.npz
!pip install pytorch-ignite tensorboardX

--2019-11-18 16:38:30--  https://raw.githubusercontent.com/cemoody/simple_mf/master/notebooks/loader.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1014 [text/plain]
Saving to: ‘loader.py’

loader.py           100%[===================>]    1014  --.-KB/s    in 0s      

2019-11-18 16:38:30 (194 MB/s) - ‘loader.py’ saved [1014/1014]

--2019-11-18 16:38:31--  https://www.dropbox.com/s/msqzmmaog5bstm4/dataset.npz?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/msqzmmaog5bstm4/dataset.npz [following]
--2019-11-18 16:38:31--  https://www.dropbox.com/s/raw/msqzmmaog5b

In [4]:
import numpy as np
fh = np.load('data/dataset.npz')

# We have a bunch of feature columns and last column is the y-target
# Note pytorch is finicky about need int64 types
train_x = fh['train_x'].astype(np.int64)
train_y = fh['train_y']

# We've already split into train & test
test_x = fh['test_x'].astype(np.int64)
test_y = fh['test_y']


n_user = int(fh['n_user'])
n_item = int(fh['n_item'])

# columns are user_id, item_id and other features 
# we won't use the 3rd and 4th columns
print(train_x)
print(' ')
print(train_y)

[[   1 1193   11   10]
 [   1  914   26   10]
 [   1 3408    7   10]
 ...
 [6040  562   37    6]
 [6040 1096  109    6]
 [6040 1097   99    6]]
 
[[5.]
 [3.]
 [4.]
 ...
 [5.]
 [4.]
 [4.]]


### Define the MF Model

In [0]:
import torch
from torch import nn
import torch.nn.functional as F

In [0]:
class MF(nn.Module):
    itr = 0
    def __init__(self, n_user, n_item, k=18, c_vector=1.0, writer=None):
        super(MF, self).__init__()
        # This will hold the logging
        self.writer = writer
        
        # These are simple hyperparameters
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.c_vector = c_vector
        
        # These are learned and fit by PyTorch
        self.user = nn.Embedding(n_user, k)
        self.item = nn.Embedding(n_item, k)
    
    def __call__(self, train_x):
        # This is the most import function in this script
        # These are the user indices, and correspond to "u" variable
        user_id = train_x[:, 0]
        # Item indices, correspond to the "i" variable
        item_id = train_x[:, 1]
        # vector user = p_u
        vector_user = self.user(user_id)
        # vector item = q_i
        vector_item = self.item(item_id)
        # this is a dot product & a user-item interaction: p_u * q_i
        ui_interaction = torch.sum(vector_user * vector_item, dim=1)
        return ui_interaction
    
    def loss(self, prediction, target):
        # MSE error between target = R_ui and prediction = p_u * q_i
        loss_mse = F.mse_loss(prediction, target.squeeze())
        # Compute L2 reularization over user (P) and item (Q) matrices
        prior_user =  l2_regularize(self.user.weight) * self.c_vector
        prior_item = l2_regularize(self.item.weight) * self.c_vector
        # Add up the MSE loss + user & item regularization
        total = loss_mse + prior_user + prior_item
        
        # This logs all local variables to tensorboard
        for name, var in locals().items():
            if type(var) is torch.Tensor and var.nelement() == 1 and self.writer is not None:
                self.writer.add_scalar(name, var, self.itr)
        return total

In [0]:
def l2_regularize(array):
    loss = torch.sum(array ** 2.0)
    return loss

### Train model

In [0]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Loss
from ignite.metrics import MeanSquaredError

from tensorboardX import SummaryWriter

from loader import Loader
from datetime import datetime

In [10]:
# Hyperparameters
lr = 1e-2
# Number of dimensions per user, item
k = 10
# regularization constant
c_vector = 1e-6

# Setup logging
log_dir = 'runs/simple_mf_01_' + str(datetime.now()).replace(' ', '_')
print(log_dir)
writer = SummaryWriter(log_dir=log_dir)

runs/simple_mf_01_2019-11-18_16:39:01.356376


In [0]:
model = MF(n_user, n_item, writer=writer, k=k, c_vector=c_vector)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

trainer = create_supervised_trainer(model, optimizer, model.loss)
metrics = {'accuracy': MeanSquaredError()}
evaluat = create_supervised_evaluator(model, metrics=metrics)

train_loader = Loader(train_x, train_y, batchsize=1024)
test_loader = Loader(test_x, test_y, batchsize=1024)

In [12]:
def log_training_loss(engine, log_interval=400):
    model.itr = engine.state.iteration
    if model.itr % log_interval == 0:
        fmt = "Epoch[{}] Iteration[{}/{}] Loss: {:.2f}"
        msg = fmt.format(engine.state.epoch, engine.state.iteration, 
                         len(train_loader), engine.state.output)
        print(msg)

trainer.add_event_handler(event_name=Events.ITERATION_COMPLETED, handler=log_training_loss)

def log_validation_results(engine):
    # When triggered, run the validation set
    evaluat.run(test_loader)
    avg_accuracy = evaluat.state.metrics['accuracy']
    print("Epoch[{}] Validation MSE: {:.2f} ".format(engine.state.epoch, avg_accuracy))
    writer.add_scalar("validation/avg_accuracy", avg_accuracy, engine.state.epoch)

trainer.add_event_handler(event_name=Events.EPOCH_COMPLETED, handler=log_validation_results)


model

MF(
  (user): Embedding(6041, 10)
  (item): Embedding(3953, 10)
)

#### Run model

In [14]:
trainer.run(train_loader, max_epochs=15)

Epoch[1] Iteration[400/879] Loss: 0.88
Epoch[1] Iteration[800/879] Loss: 0.94
Epoch[1] Validation MSE: 0.91 
Epoch[2] Iteration[1200/879] Loss: 0.89
Epoch[2] Iteration[1600/879] Loss: 0.90
Epoch[2] Validation MSE: 0.88 
Epoch[3] Iteration[2000/879] Loss: 0.84
Epoch[3] Iteration[2400/879] Loss: 0.87
Epoch[3] Validation MSE: 0.86 
Epoch[4] Iteration[2800/879] Loss: 0.85
Epoch[4] Iteration[3200/879] Loss: 0.84
Epoch[4] Validation MSE: 0.84 
Epoch[5] Iteration[3600/879] Loss: 0.76
Epoch[5] Iteration[4000/879] Loss: 0.77
Epoch[5] Validation MSE: 0.82 
Epoch[6] Iteration[4400/879] Loss: 0.73
Epoch[6] Iteration[4800/879] Loss: 0.78
Epoch[6] Iteration[5200/879] Loss: 0.83
Epoch[6] Validation MSE: 0.82 
Epoch[7] Iteration[5600/879] Loss: 0.74
Epoch[7] Iteration[6000/879] Loss: 0.79
Epoch[7] Validation MSE: 0.82 
Epoch[8] Iteration[6400/879] Loss: 0.78
Epoch[8] Iteration[6800/879] Loss: 0.80
Epoch[8] Validation MSE: 0.81 
Epoch[9] Iteration[7200/879] Loss: 0.78
Epoch[9] Iteration[7600/879] Loss:

In [15]:
model.user.weight.data.numpy()

array([[-2.64752784e-38, -1.53490092e-38,  1.03375581e-38, ...,
        -2.70205236e-38, -1.11510848e-39,  1.06727529e-38],
       [-2.07549483e-01,  3.31319749e-01,  1.42050898e+00, ...,
        -7.58199513e-01,  1.04956484e+00,  4.61411357e-01],
       [ 4.51680601e-01, -8.31330359e-01,  1.36043549e+00, ...,
        -7.76251376e-01,  8.32142532e-02, -3.88718490e-03],
       ...,
       [ 5.92992961e-01, -8.98332357e-01,  1.31741285e+00, ...,
        -6.48082793e-01, -8.40339601e-01, -3.27048391e-01],
       [ 6.51513040e-01, -1.23964697e-01,  8.29760790e-01, ...,
         4.34264019e-02, -1.20709784e-01, -2.47810379e-01],
       [ 4.56198692e-01,  2.68148094e-01,  1.38877046e+00, ...,
         5.33053219e-01, -6.23601794e-01, -9.39349651e-01]], dtype=float32)